<a href="https://colab.research.google.com/github/Nico-lly/qd_cegonha/blob/analise_exploratoria/1_Sele%C3%A7%C3%A3o_de_vari%C3%A1veis_constru%C3%A7%C3%A3o_de_index_e_consolida%C3%A7%C3%A3o_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações de bibliotecas

In [2]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.5 MB/s eta 0:00:00


In [53]:
from unidecode import unidecode
import pandas as pd
import numpy as np
import re
import os

# Montar drive

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Seleção e conformação prévia de dados

## Data SUS

In [6]:
# DEFININDO diretorio Data_SUS
PATH = os.path.abspath(r'/content/drive/MyDrive/bases_rede_cegonha_OKBR/Data_SUS')

files = os.listdir(PATH)
filepaths = [os.path.join(PATH, file) for file in files if re.search(r'DATASUS_SANEAMENTO', file)]

filepaths = filepaths[::1]

### Saneamento

In [7]:
saneamento_nacional_df = pd.DataFrame()
saneamento_municipal_df = pd.DataFrame()
anos_lista = ['2010', '2011', '2012', '2013', '2014', '2015']

for i_arquivo, arquivo_ano in enumerate(filepaths):
  #Ler o arquivo
  df_ano = pd.read_csv(arquivo_ano, encoding = 'latin-1', skiprows=[0, 1, 2], sep = ';')

  #Criar novas colunas
  df_ano['cod_municipio'] = pd.Series()
  df_ano['nome_municipio'] = pd.Series()
  df_ano['ano'] = anos_lista[i_arquivo]

  #Preencer colunas separadas de codigo e nome de município
  for i_municipio, infos_unidas in enumerate(df_ano['Município']):
    municipio_split = infos_unidas.split(' ')
    df_ano.loc[i_municipio, 'cod_municipio'] = municipio_split[0]
    df_ano.loc[i_municipio, 'nome_municipio'] = ' '.join(municipio_split[1:])

  #Excluindo coluna de municipio antiga e a ultima linha sem dados
  df_ano.drop(['Município'], axis = 1, inplace = True)
  df_ano.drop(df_ano.index[-1], axis = 0, inplace = True)

  #Separa a linha total para df nacional
  saneamento_nacional_df = pd.concat([saneamento_nacional_df, df_ano.iloc[[-1]]], ignore_index=True)

  #Separa linhas para df municipal
  saneamento_municipal_df = pd.concat([saneamento_municipal_df, df_ano[:-1]], ignore_index=True)

<ipython-input-7-b522402d5640>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['cod_municipio'] = pd.Series()
<ipython-input-7-b522402d5640>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['nome_municipio'] = pd.Series()
<ipython-input-7-b522402d5640>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['cod_municipio'] = pd.Series()
<ipython-input-7-b522402d5640>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['nome_municipio'] = pd.Series()
<ipython-input-7-b522402d5640>:10: FutureWarning: The 

In [8]:
# CONCATENANDO INFORMAÇÕES
saneamento_municipal_df['abastecimento_agua'] = saneamento_municipal_df[['Ab_água-rede_públ', 'Ab_água-poço/nasc.', 'Ab_água-outros']].replace('-', np.NaN).astype('float').sum(axis = 1)
saneamento_municipal_df['tratamento_agua'] = saneamento_municipal_df[['Trat.Água_Filtrada', 'Trat.Água_Fervida', 'Trat.Água_Clorada']].replace('-', np.NaN).astype('float').sum(axis = 1)
saneamento_municipal_df['esgotamento'] = saneamento_municipal_df[['Fez.Urina-esgoto', 'Fez.Urina-fossa']].replace('-', np.NaN).astype('float').sum(axis = 1)
saneamento_municipal_df['coleta_lixo'] = saneamento_municipal_df['Lixo-coletado']
saneamento_municipal_df['tratamento_lixo'] = saneamento_municipal_df[['Lixo-queim/enterr']].replace('-', np.NaN).astype('float').sum(axis = 1)
saneamento_municipal_df['bolsa_familia'] = saneamento_municipal_df['Fam.Benef._do_PBF'].replace('-', np.NaN).astype('float')
saneamento_municipal_df['cad_unico'] = saneamento_municipal_df['Fam._com_CADÚnico'].replace('-', np.NaN).astype('float')
saneamento_municipal_df.rename(columns = {'Nº_Famílias':'qtd_familias'}, inplace = True)

In [9]:
#DROPANDO COLUNAS DESNECESSARIAS
saneamento_municipal_df = saneamento_municipal_df.drop(saneamento_municipal_df[saneamento_municipal_df['cod_municipio'] == 'Total'].index)
saneamento_municipal_df = saneamento_municipal_df.drop(saneamento_municipal_df[saneamento_municipal_df['nome_municipio'].str.contains('MUNICIPIO IGNORADO')].index)

In [10]:
# DEFININDO DATA FRAME PRINCIPAL E SECUNDÁRIO
df_saneamento_principal = saneamento_municipal_df[['cod_municipio', 'nome_municipio', 'ano', 'qtd_familias', 'abastecimento_agua', 'tratamento_agua', 'esgotamento', 'coleta_lixo', 'tratamento_lixo']]
df_saneamento_secundario = saneamento_municipal_df[['bolsa_familia', 'cad_unico',]]

In [11]:
#TRANSFORMANDO EM INTEIRO
df_saneamento_principal['abastecimento_agua'] = df_saneamento_principal['abastecimento_agua'].apply(lambda x: int(x))
df_saneamento_principal['tratamento_agua'] = df_saneamento_principal['tratamento_agua'].apply(lambda x: int(x))
df_saneamento_principal['esgotamento'] = df_saneamento_principal['esgotamento'].apply(lambda x: int(x))
df_saneamento_principal['coleta_lixo'] = df_saneamento_principal['coleta_lixo'].apply(lambda x: int(x.replace('-', '').replace('', '0')))
df_saneamento_principal['tratamento_lixo'] = df_saneamento_principal['tratamento_lixo'].apply(lambda x: int(x))

<ipython-input-11-b0760427a1ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_saneamento_principal['abastecimento_agua'] = df_saneamento_principal['abastecimento_agua'].apply(lambda x: int(x))
<ipython-input-11-b0760427a1ab>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_saneamento_principal['tratamento_agua'] = df_saneamento_principal['tratamento_agua'].apply(lambda x: int(x))
<ipython-input-11-b0760427a1ab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [12]:
df_saneamento_principal.tail(2)

,cod_municipio,nome_municipio,ano,qtd_familias,abastecimento_agua,tratamento_agua,esgotamento,coleta_lixo,tratamento_lixo
30203,522230,VILA PROPICIO,2015,1529.0,1529,1225,1461,608020,811
30204,530010,BRASILIA,2015,186040.0,186040,170198,185517,107040502040,8033


### Cobertura pré-natal

In [13]:
cobertura_prenatal_file = "/content/drive/MyDrive/bases_rede_cegonha_OKBR/Data_SUS/DATASUS_COBERTURAPRENATAL_1998_2015.csv"

In [14]:
df_prenatal = pd.read_csv(cobertura_prenatal_file, encoding = 'latin1', skiprows=[0, 1, 2], sep = ';')

In [15]:
# Retirar linhas sem utilidade
df_prenatal = df_prenatal.iloc[:-2]
df_prenatal = df_prenatal.drop(df_prenatal[df_prenatal['Município'].str.contains('MUNICIPIO IGNORADO')].index)
#df_prenatal = df_prenatal.drop(df_prenatal[df_prenatal['cod_municipio'] == 'Total'].index)

#Retirar coluna "total"
df_prenatal.drop(['Total'], axis = 1, inplace = True)

In [16]:
# Lista das colunas de ano
colunas_ano = df_prenatal.columns[1:]

# Crie um novo DataFrame 'df_melt' usando melt
df_prenatal = pd.melt(df_prenatal, id_vars=['Município'], value_vars=colunas_ano, var_name='ano', value_name='prenatal')

In [17]:
#Separando o codigo do municipio do nome do municipio
df_prenatal['cod_municipio'] = pd.Series()
df_prenatal['nome_municipio'] = pd.Series()
for i_municipio, infos_unidas in enumerate(df_prenatal['Município']):
    municipio_split = infos_unidas.split(' ')
    df_prenatal.loc[i_municipio, 'cod_municipio'] = municipio_split[0]
    df_prenatal.loc[i_municipio, 'nome_municipio'] = ' '.join(municipio_split[1:])

<ipython-input-17-beae0f4a63b4>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_prenatal['cod_municipio'] = pd.Series()
<ipython-input-17-beae0f4a63b4>:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_prenatal['nome_municipio'] = pd.Series()


In [18]:
# Retirar coluna antiga de municipio
df_prenatal.drop(['Município'], axis = 1, inplace = True)

In [19]:
#TRANSFORMAR EM INTEIROS
df_prenatal['prenatal'] = df_prenatal['prenatal'].apply(lambda x: int(x.replace('-', '').replace('', '0')))

### Óbito materno

In [20]:
filepaths = [os.path.join(PATH, file) for file in files if re.search(r'datasus_obitos_maternos', file)]

In [21]:
df_obito_materno = pd.DataFrame()
anos_lista = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

for i_arquivo, arquivo_ano in enumerate(filepaths):
  #Ler o arquivo
  df_ano = pd.read_csv(arquivo_ano, encoding = 'latin-1', skiprows=[0, 1, 2], sep = ';')
  df_ano = df_ano.iloc[0:-16]

  #Criar novas colunas
  df_ano['cod_municipio'] = pd.Series()
  df_ano['nome_municipio'] = pd.Series()
  df_ano['ano'] = anos_lista[i_arquivo]

  #Preencer colunas separadas de codigo e nome de município
  for i_municipio, infos_unidas in enumerate(df_ano['Município']):
    municipio_split = infos_unidas.split(' ')
    df_ano.loc[i_municipio, 'cod_municipio'] = municipio_split[0]
    df_ano.loc[i_municipio, 'nome_municipio'] = ' '.join(municipio_split[1:])

  #Excluindo coluna de municipio antiga e a ultima linha sem dados
  df_ano.drop(['Município'], axis = 1, inplace = True)
  #df_ano.drop(df_ano.index[-1], axis = 0, inplace = True)

  #Separa a linha total para df nacional
  df_obito_materno = pd.concat([df_obito_materno, df_ano], ignore_index=True)

<ipython-input-21-b143358f2c04>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['cod_municipio'] = pd.Series()
<ipython-input-21-b143358f2c04>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['nome_municipio'] = pd.Series()
<ipython-input-21-b143358f2c04>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['cod_municipio'] = pd.Series()
<ipython-input-21-b143358f2c04>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_ano['nome_municipio'] = pd.Series()
<ipython-input-21-b143358f2c04>:10: FutureWarning:

In [22]:
df_obito_materno['obito_materno'] = df_obito_materno['Óbitos_maternos'] + df_obito_materno['Óbitos_maternos_tardios']
df_obito_materno = df_obito_materno[['obito_materno', 'cod_municipio', 'nome_municipio', 'ano']]

In [23]:
#DROPAR LINHAS DESNCESSÁRIAS
df_obito_materno = df_obito_materno.drop(df_obito_materno[df_obito_materno['cod_municipio'] == 'Total'].index)
df_obito_materno = df_obito_materno.drop(df_obito_materno[df_obito_materno['nome_municipio'].str.contains('MUNICIPIO IGNORADO')].index)

In [24]:
#TRANSFORMAR EM INTEIROS
df_obito_materno['obito_materno'] = df_obito_materno['obito_materno'].apply(lambda x: int(x.replace('-', '').replace('', '0')))

In [25]:
df_obito_materno.tail()

,obito_materno,cod_municipio,nome_municipio,ano
54077,10,522185,VALPARAISO DE GOIAS,2021
54078,0,522190,VARJAO,2021
54079,0,522200,VIANOPOLIS,2021
54080,0,522205,VICENTINOPOLIS,2021
54081,0,522220,VILA BOA,2021



## PCDaS

### Óbitos fetais


In [26]:
# DEFININDO diretorio PCDaS - óbitos Fetais
PATH = os.path.abspath(r'/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020')

files = os.listdir(PATH)
filepaths = [os.path.join(PATH, file) for file in files if re.search(r'ETLSIM.DOFET', file)]

#filepaths = filepaths[::1]
print(filepaths)

['/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2002_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2003_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2004_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_1999_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2011_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2019_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2005_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2008_t.csv', '/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_OBITOSFETAIS_1996_2020/ETLSIM.DOFET_2006_t.csv', '/content/drive/My

In [27]:
anos_lista = [1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
df_obitosfetais = pd.DataFrame()

for i_ano, ano_arquivo in enumerate(filepaths):
    df_ano = pd.read_csv(ano_arquivo, encoding = 'latin1', sep = ',')
    df_ano['ano'] = anos_lista[i_ano]
    df_obitosfetais = pd.concat([df_obitosfetais, df_ano])

<ipython-input-27-99fcb4cc09bc>:5: DtypeWarning: Columns (89) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(ano_arquivo, encoding = 'latin1', sep = ',')
<ipython-input-27-99fcb4cc09bc>:5: DtypeWarning: Columns (32,35,36,37,38,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(ano_arquivo, encoding = 'latin1', sep = ',')
<ipython-input-27-99fcb4cc09bc>:5: DtypeWarning: Columns (90) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(ano_arquivo, encoding = 'latin1', sep = ',')
<ipython-input-27-99fcb4cc09bc>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(ano_arquivo, encoding = 'latin1', sep = ',')
<ipython-input-27-99fcb4cc09bc>:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(ano_arquivo

In [28]:
#SELECIONANDO COLUNAS
df_obitosfetais = df_obitosfetais[['ESCMAE2010', 'def_raca_cor', 'CODMUNOCOR', 'OBITOPARTO', 'FONTE', 'PARTO',
                                   'def_assist_med', 'def_sexo', 'res_MUNNOMEX', 'res_SIGLA_UF', 'CAUSABAS', 'ano']]

In [29]:
#MODIFICANDO NOMES DE COLUNAS
df_obitosfetais.rename(columns = {'ESCMAE2010': 'escolaridade_mae', 'def_raca_cor': 'raca_cor', 'CODMUNOCOR':'cod_municipio',
                                  'OBITOPARTO': 'obito_parto', 'FONTE':'fonte', 'PARTO': 'tipo_parto', 'def_assist_med':'assist_medica', 'def_sexo':'sexo', 'res_MUNNOMEX':'nome_municipio',
                                  'res_SIGLA_UF':'sigla_uf', 'CAUSABAS':'causa_mortis'}, inplace = True)

<ipython-input-29-e89c19acb196>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obitosfetais.rename(columns = {'ESCMAE2010': 'escolaridade_mae', 'def_raca_cor': 'raca_cor', 'CODMUNOCOR':'cod_municipio',


In [30]:
df_obitosfetais['obito_parto'] = df_obitosfetais['obito_parto'].replace({'1':'antes','2':'durante','3':'depois', '9':'ignorado'})
df_obitosfetais['raca_cor'] = df_obitosfetais['raca_cor'].replace({'Ignorado':'ignorado', 'Branca':'branca', 'Parda':'parda',
                                                                   'Preta':'preta', 'IndÃ\xadgena':'indigena', 'Amarela':'amarela'})


<ipython-input-30-48a234884be9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_obitosfetais['obito_parto'] = df_obitosfetais['obito_parto'].replace({'1':'antes','2':'durante','3':'depois', '9':'ignorado'})


In [31]:
df_obitosfetais_principal = df_obitosfetais.groupby(['cod_municipio', 'nome_municipio', 'sigla_uf', 'ano']).size().reset_index()
df_obitosfetais_principal.columns = ['cod_municipio', 'nome_municipio', 'sigla_uf', 'ano', 'obitos_fetais']

In [32]:
#DROPAR LINHAS DESNCESSÁRIAS
df_obitosfetais_principal = df_obitosfetais_principal.drop(df_obitosfetais_principal[df_obitosfetais_principal['cod_municipio'] == 'Total'].index)

In [33]:
df_obitosfetais_secundario = df_obitosfetais

In [34]:
df_obitosfetais_principal.tail()

,cod_municipio,nome_municipio,sigla_uf,ano,obitos_fetais
166869,5300108,VILA BOA,GO,1998,1
166870,5300108,VILA BOA,GO,2005,2
166871,5300108,VILA BOA,GO,2011,1
166872,5300108,VILA BOA,GO,2013,1
166873,5300108,VILA PROPICIO,GO,2011,1


### Polis (dados de apoio)

In [35]:
polis_file = "/content/drive/MyDrive/bases_rede_cegonha_OKBR/PCDaS/PCDAS_POLIS_2018_19_21.csv"
df_polis = pd.read_csv(polis_file, encoding = 'latin1', sep = ',')

<ipython-input-35-74e380d41b43>:2: DtypeWarning: Columns (123,124,125,126,127,129,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df_polis = pd.read_csv(polis_file, encoding = 'latin1', sep = ',')


In [36]:
df_polis.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 227 columns):
 #    Column                                  Dtype  
---   ------                                  -----  
 0    Unnamed: 0                              int64  
 1    COD_IBGE                                int64  
 2    POPULACAO_2019                          int64  
 3    qtd_obitos_ate_um_ano_2019              float64
 4    qtd_NASC_2019                           float64
 5    qtd_nasc_7_mais_consultas_2019          float64
 6    qtd_nasc_baixo_peso_2019                float64
 7    qtd_OBITOS_2019                         int64  
 8    qtd_obitos_homicidio_2019               float64
 9    qtd_obitos_acidente_2019                float64
 10   qtd_obitos_suicidios_2019               float64
 11   qtd_obitos_agressoes_2019               float64
 12   qtd_obitos_maternos_2019                float64
 13   qtd_nasc_maes_10_14_2019                float64
 14   qtd_nasc_maes_15_17_20

## População feminina IBGE

In [37]:
anos_lista = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
PATH = '/content/drive/MyDrive/bases_rede_cegonha_OKBR/IBGE/Populacao feminina/pop_fem_10_49_anos'
files = os.listdir(PATH)

In [38]:
def years_sum(year, files, path):
  #Selecionar arquivos de um ano especifico
  filepath = [os.path.join(path, file) for file in files if re.search(re.compile(year), file)]

  # Criar o df de saída
  df_total_ano = pd.DataFrame(columns = ['Município', 'População_residente'])

  for file in filepath:
    #Ler arquivos
    df_ano = pd.read_csv(file, encoding = 'latin-1', skiprows=[0, 1, 2, 3, 4], sep = ';')

    #Excluir ultimas linhas
    df_ano = df_ano.iloc[:-3]
    df_ano = df_ano.drop(df_ano[df_ano['Município'].str.contains('MUNICIPIO IGNORADO')].index)

    df_total_ano = pd.concat([df_ano, df_total_ano], ignore_index=True)

  #Criar novas colunas
  df_total_ano['cod_municipio'] = pd.Series()
  df_total_ano['nome_municipio'] = pd.Series()
  df_total_ano['ano'] = year

  #Preencer colunas separadas de codigo e nome de município
  for i_municipio, infos_unidas in enumerate(df_total_ano['Município']):
    municipio_split = infos_unidas.split(' ')
    df_total_ano.loc[i_municipio, 'cod_municipio'] = municipio_split[0]
    df_total_ano.loc[i_municipio, 'nome_municipio'] = ' '.join(municipio_split[1:])

  #Excluindo coluna de municipio antiga
  df_total_ano.drop(['Município'], axis = 1, inplace = True)

  df_total_ano = df_total_ano.groupby(['cod_municipio', 'nome_municipio', 'ano'], as_index=False)['População_residente'].sum()


  return df_total_ano

In [39]:
df_populacao_fem = pd.DataFrame(columns = ['População_residente', 'cod_municipio', 'nome_municipio', 'ano'])
for ano in anos_lista:
  df_ano = years_sum(ano, files, PATH)
  df_populacao_fem = pd.concat([df_populacao_fem, df_ano])

<ipython-input-38-651d4896629e>:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_total_ano['cod_municipio'] = pd.Series()
<ipython-input-38-651d4896629e>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_total_ano['nome_municipio'] = pd.Series()
<ipython-input-38-651d4896629e>:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_total_ano['cod_municipio'] = pd.Series()
<ipython-input-38-651d4896629e>:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_total_ano['nome_municipio'] = pd.Series()
<ipython-input-38-651d4896

In [40]:
df_populacao_fem.rename(columns = {'População_residente' : 'populacao_feminina'}, inplace = True)

In [41]:
#DROPAR LINHAS DESNCESSÁRIAS
df_populacao_fem = df_populacao_fem.drop(df_populacao_fem[df_populacao_fem['cod_municipio'] == 'Total'].index)
df_populacao_fem = df_populacao_fem.drop(df_populacao_fem[df_populacao_fem['nome_municipio'].str.contains('MUNICIPIO IGNORADO')].index)

In [42]:
df_populacao_fem.tail()

,populacao_feminina,cod_municipio,nome_municipio,ano
5565,4255.0,522200,VIANOPOLIS,2021
5566,2602.0,522205,VICENTINOPOLIS,2021
5567,1712.0,522220,VILA BOA,2021
5568,1610.0,522230,VILA PROPICIO,2021
5569,1014301.0,530010,BRASILIA,2021


## População total estimada

In [44]:
anos_lista = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
PATH = '/content/drive/MyDrive/bases_rede_cegonha_OKBR/IBGE/Estimativa populacional'
files = os.listdir(PATH)

In [95]:
ano = '2011'
arquivo_pop_ano = PATH + f'/estimativa_{ano}.xls'
df_pop_ano = pd.read_excel(arquivo_pop_ano, skiprows= (1), header = None)
df_pop_ano = df_pop_ano.iloc[2:33, :]
df_pop_ano = df_pop_ano.iloc[:, 3:5]
df_pop_ano.head()

,3,4
2,Alta Floresta D'Oeste,24228
3,Alto Alegre dos Parecis,12824
4,Alto Paraíso,17443
5,Alvorada D'Oeste,16624
6,Ariquemes,91570


In [93]:
df_populacao = pd.DataFrame(columns = ['uf', 'pop_estimada', 'ano'])

for ano in anos_lista:

  arquivo_pop_ano = PATH + f'/estimativa_{ano}.xls'

  if ano < '2017':
    df_pop_ano = pd.read_excel(arquivo_pop_ano, skiprows= (1), header = None)
    df_pop_ano = df_pop_ano.iloc[2:33, :]
    df_pop_ano = df_pop_ano.iloc[:, 3:4]
    #df_pop_ano.columns = ['uf', 'pop_estimada']

  elif ano >= '2017':
    df_pop_ano = pd.read_excel(arquivo_pop_ano, skiprows= (1))
    df_pop_ano = df_pop_ano.iloc[1:33, :]
    df_pop_ano = df_pop_ano[['BRASIL E UNIDADES DA FEDERAÇÃO', 'POPULAÇÃO ESTIMADA']]
    #df_pop_ano.rename(columns = {'BRASIL E UNIDADES DA FEDERAÇÃO': 'uf', 'POPULAÇÃO ESTIMADA': 'pop_estimada'}, inplace = True)

  df_pop_ano.columns = ['uf', 'pop_estimada']
  df_pop_ano['ano'] = ano
  id_drop = df_pop_ano[df_pop_ano['uf'].str.contains('região', case=False, na=False)].index
  df_pop_ano.drop(id_drop, inplace = True)
  df_pop_ano['uf'] = df_pop_ano['uf'].apply(lambda x: unidecode(x.upper()))

  df_populacao = pd.concat([df_pop_ano, df_populacao], ignore_index=True)

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [54]:
df_populacao['ano'].unique()

,uf,pop_estimada
2,RONDONIA,1815278
3,ACRE,906876
4,AMAZONAS,4269995
5,RORAIMA,652713
6,PARA,8777124


# Unificação de dataframe

In [ ]:
colunas_para_int = ['cod_municipio', 'ano']
dataframes = [df_obitosfetais_principal, df_saneamento_principal, df_prenatal, df_obito_materno, df_populacao_fem]

for dataframe in dataframes:
    for coluna in colunas_para_int:
      dataframe[coluna] = pd.to_numeric(dataframe[coluna]).astype(int)

    # Filtragem após a conversão para inteiros
    dataframes = dataframe[dataframe['ano'] >= 2011]

In [ ]:
resultado = pd.merge(df_populacao_fem, df_saneamento_principal, on=['ano','nome_municipio', 'cod_municipio'], how='left')
resultado = pd.merge(resultado, df_prenatal, on=['ano', 'nome_municipio', 'cod_municipio'], how='left')
resultado = pd.merge(resultado, df_obito_materno, on=['ano', 'nome_municipio', 'cod_municipio'], how='left')
resultado = pd.merge(resultado, df_obitosfetais_principal, on=['ano', 'nome_municipio', 'cod_municipio'], how='left')

# Ajustes de dataframe unificado

### Inclusão de estados por município

In [ ]:
uf_municipios_file = "/content/drive/MyDrive/bases_rede_cegonha_OKBR/IBGE/Municipios e estados/RELATORIO_DTB_BRASIL_MUNICIPIO.xls"
df_uf_municipios = pd.read_excel(uf_municipios_file,  skiprows=[0, 1, 2, 3, 4, 5])
df_uf_municipios = df_uf_municipios[['Nome_UF', 'UF', 'Nome_Município']]
df_uf_municipios.rename(columns = {'Nome_UF': 'uf', 'UF': 'cod_uf', 'Nome_Município':'nome_municipio'}, inplace = True)

In [ ]:
df_uf_municipios['nome_municipio'] = df_uf_municipios['nome_municipio'].apply(lambda x: unidecode.unidecode(x.upper()))
df_uf_municipios['uf'] = df_uf_municipios['uf'].apply(lambda x: unidecode.unidecode(x.upper()))

In [ ]:
resultado['cod_uf'] = resultado['cod_municipio'].apply(lambda x: int(str(x)[:2]))

In [ ]:
resultado = pd.merge(resultado, df_uf_municipios, on=['nome_municipio', 'cod_uf'], how='left')

In [ ]:
resultado.head()

### Inclusão de região por uf

In [ ]:
def atribuir_regiao(estado):
  norte = ['ACRE', 'AMAPA', 'AMAZONAS', 'PARA', 'RONDONIA', 'RORAIMA', 'TOCANTINS']
  nordeste = ['ALAGOAS', 'BAHIA', 'CEARA', 'MARANHAO', 'PARAIBA', 'PERNAMBUCO', 'PIAUI', 'RIO GRANDE DO NORTE', 'SERGIPE']
  centro_oeste = ['DISTRITO FEDERAL', 'GOIAS', 'MATO GROSSO', 'MATO GROSSO DO SUL']
  sudeste = ['ESPIRITO SANTO', 'MINAS GERAIS', 'RIO DE JANEIRO', 'SAO PAULO']
  sul = ['PARANA', 'RIO GRANDE DO SUL', 'SANTA CATARINA']

  if estado in norte:
    return 'NORTE'
  elif estado in nordeste:
    return 'NORDESTE'
  elif estado in centro_oeste:
    return 'CENTRO-OESTE'
  elif estado in sudeste:
    return 'SUDESTE'
  elif estado in sul:
    return 'SUL'
  else:
    return 'Região não especificada'

In [ ]:
resultado['regiao'] = resultado['uf'].apply(atribuir_regiao)

In [ ]:
resultado['ano'].unique()

# Criação de índices


In [ ]:
###Chaves para concatenação###
#Nacional: ano
#Estado: 2 primeiros caracteres do municipio
#Municipal: Codigo do municipio e ano

## Analisar
# - Quartis e outliers
# - Maior e menor para cada Grupo

In [ ]:
# CRIANDO ÍNDICES

####SANEAMENTO#####
# validar com levantamento de população (quantidade de familias)
# criar indice rankeado para saneamento por municipio (acima da media brasileira)
# separar bolsa familia

####COBERTURA PRE NATAL####
# comparar população residente do sexo feminino por ano e por municipio 18-40 anos (quantos porcento)

####OBITO FETAL####
#Criar ranking de 1 a 5 para obito fetal

####OBITO MATERNO####
#Criar ranking de 1 a 5 para obito fetal

In [ ]:
resultado.columns

In [ ]:
#1 - Mortalidade e Morbidade
#IMM = 'obito_materno' + 'obitos_fetais' / 'populacao_residente'

#2 - Atenção
#ICP = 'prenatal' / 'populacao_residente'

#3 - Capacidade hospitalar instalada
# IAPS = 'qtd_familias' / 'populacao_residente'

#4 - Gestão
#ISB = ('abastecimento_agua' + 'tratamento_agua' + 'esgotamento' + 'coleta_lixo' + 'tratamento_lixo' / 5) / 'populacao_residente'